## 0. Import

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from helpers import Residual, CombineDataset

## 1. Building the data set

In [2]:
# Build the four classes 
config_repo_img = 'DeeplenstronomyDataset/ConfigFile/config_img'
config_repo_model = 'DeeplenstronomyDataset/ConfigFile/config_model'
size = 10

for i in np.arange(1,4):
    img_name = config_repo_img + str(i) + '.yaml'
    model_name = config_repo_model + str(i) + '.yaml'
    res = Residual(img_name,model_name,size)
    res.build(i)

In [3]:
# Open and concatenate the metadata in one data frame
data_repo = 'DeeplenstronomyDataset/DataSet/'
metadata = pd.DataFrame()
for i in np.arange(1,4):
    file_name = data_repo+'MetaE'+str(i)+'.csv'
    meta = pd.read_csv(file_name)
    metadata = pd.concat([metadata,meta], ignore_index=True)

# Remove fixed data from the metadata - !!!this may change if the configuration file is updated!!!
metadata = metadata.drop(['OBJID-g', 'H0-g', 'Om0-g', 'exposure_time-g', 'numPix-g',
                          'pixel_scale-g', 'psf_type-g', 'read_noise-g', 'ccd_gain-g', 'seeing-g',
                          'magnitude_zero_point-g', 'sky_brightness-g', 'num_exposures-g',
                          'NUMBER_OF_NOISE_SOURCES-g', 'CONFIGURATION_LABEL-g',
                          'CONFIGURATION_NAME-g', 'NUMBER_OF_PLANES-g','PLANE_1-NUMBER_OF_OBJECTS-g', 'PLANE_1-OBJECT_1-NAME-g',
                          'PLANE_2-NUMBER_OF_OBJECTS-g', 'PLANE_2-OBJECT_1-NAME-g', 'PLANE_1-OBJECT_1-REDSHIFT-g',
                          'PLANE_1-OBJECT_1-NUMBER_OF_LIGHT_PROFILES-g','PLANE_1-OBJECT_1-NUMBER_OF_SHEAR_PROFILES-g',
                          'PLANE_1-OBJECT_1-NUMBER_OF_MASS_PROFILES-g', 'PLANE_1-OBJECT_1-HOST-g',
                          'PLANE_1-OBJECT_1-LIGHT_PROFILE_1-NAME-g', 'PLANE_1-OBJECT_1-LIGHT_PROFILE_1-magnitude-g',
                          'PLANE_1-OBJECT_1-LIGHT_PROFILE_1-center_x-g',
                          'PLANE_1-OBJECT_1-LIGHT_PROFILE_1-center_y-g', 'PLANE_2-OBJECT_1-REDSHIFT-g',
                          'PLANE_2-OBJECT_1-NUMBER_OF_LIGHT_PROFILES-g', 'PLANE_2-OBJECT_1-NUMBER_OF_SHEAR_PROFILES-g',
                          'PLANE_2-OBJECT_1-NUMBER_OF_MASS_PROFILES-g', 'PLANE_2-OBJECT_1-HOST-g',
                          'PLANE_1-OBJECT_1-MASS_PROFILE_1-center_x-g', 'PLANE_1-OBJECT_1-MASS_PROFILE_1-center_y-g',
                          'PLANE_2-OBJECT_1-LIGHT_PROFILE_1-NAME-g', 'PLANE_2-OBJECT_1-LIGHT_PROFILE_1-magnitude-g',
                          'PLANE_2-OBJECT_1-LIGHT_PROFILE_1-center_x-g', 'PLANE_2-OBJECT_1-LIGHT_PROFILE_1-center_y-g', 
                          'BACKGROUND_IDX-g', 'PLANE_1-REDSHIFT-g', 'PLANE_2-REDSHIFT-g', 'PLANE_1-OBJECT_1-LIGHT_PROFILE_1-R_sersic-g',
                          'PLANE_1-OBJECT_1-LIGHT_PROFILE_1-n_sersic-g', 'PLANE_1-OBJECT_1-LIGHT_PROFILE_1-e1-g',
                          'PLANE_1-OBJECT_1-LIGHT_PROFILE_1-e2-g','PLANE_1-OBJECT_1-MASS_PROFILE_1-NAME-g'],axis=1)

In [4]:
# Combine the residuals with the metadata in a pytorch tensor
train_set = CombineDataset(metadata,'ID','class',"DeeplenstronomyDataset/DataSet/")

loader_trainer = DataLoader(
    train_set,
    batch_size = 2,
    shuffle = True,
    num_workers = 0,
    drop_last=True
)

In [5]:
for data in loader_trainer:
    x,z, y = data
    
    print('Image Loaded Size = ', x.shape)
    print('Tabular data = ', z)
    print('Label = ', y)

Image Loaded Size =  torch.Size([2, 1, 64, 64])
Tabular data =  tensor([[ 1.0000,  0.1000, -0.1000, 18.6580,  7.8163, -0.4546,  0.6038],
        [34.1026, -0.6140,  0.1072,  6.0000,  5.0000,  0.2000, -0.1000]])
Label =  tensor([2., 1.])
Image Loaded Size =  torch.Size([2, 1, 64, 64])
Tabular data =  tensor([[ 3.2770e+01, -2.7924e-01,  8.6547e-01,  4.5476e+01,  5.3892e+00,
          2.8914e-01,  3.9582e-01],
        [ 1.0000e+00,  1.0000e-01, -1.0000e-01,  2.2586e-02,  6.7968e+00,
         -5.9491e-02,  5.1946e-01]])
Label =  tensor([3., 2.])
Image Loaded Size =  torch.Size([2, 1, 64, 64])
Tabular data =  tensor([[ 1.0000,  0.1000, -0.1000,  9.6492,  5.7593,  0.6102, -0.4690],
        [32.7702, -0.2792,  0.8655, 45.4756,  5.3892,  0.2891,  0.3958]])
Label =  tensor([2., 0.])
Image Loaded Size =  torch.Size([2, 1, 64, 64])
Tabular data =  tensor([[ 4.4844,  0.7602,  0.0505,  5.8412,  6.7960, -0.3754, -0.6076],
        [40.2562, -0.4690,  0.6067,  6.0000,  5.0000,  0.2000, -0.1000]])
Labe

## Building neural network

In [6]:
# Definition of the type of classes
classes = ('NoError', 'MassError', 'SourceError', 'SourceMassError')

In [33]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 7, 6)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(7, 16, 6)
        self.fc1 = nn.Linear(16 * 12 * 12, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        print(x.shape)
        x = x.view(2*12*12, 16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [26]:
net.forward(inputs)

torch.Size([2, 16, 12, 12])


tensor([[ 0.0135, -0.0862, -0.0782, -0.0436],
        [-0.0518, -0.1561, -0.0922, -0.0438],
        [-0.0252, -0.0988, -0.0376, -0.0533],
        [ 0.0049, -0.1259, -0.0761, -0.0223],
        [ 0.0043, -0.0843, -0.0704, -0.0505],
        [-0.0242, -0.1547, -0.0940, -0.0389],
        [-0.0283, -0.1049, -0.0487, -0.0709],
        [ 0.0108, -0.1073, -0.0541, -0.0113]], grad_fn=<AddmmBackward>)

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [35]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_trainer, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, _ ,labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

torch.Size([2, 16, 12, 12])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (288x16 and 2304x120)